In [1]:
from mmdet3d.apis import LidarDet3DInferencer

In [2]:
# initialize inferencer
# CHANGE THE center_mode in loca_visualizer accordingly (KITTI -> center_mode = 'lidar_bottom', CUSTOM -> center_mode = whatever)

#inferencer = LidarDet3DInferencer('pointpillars_donaset-car')
inferencer = LidarDet3DInferencer('pointpillars_kittilidar-car') 
# inferencer = LidarDet3DInferencer("pointpillars_hv_secfpn_8xb6-160e_kittilidar-3d-car",scope="mmdet3d")

ValueError: Cannot find model: pointpillars_kittilidar-car in mmdet3d

## Prediction Tests

In [3]:
import inspect
args = inspect.getfullargspec(inferencer.visualize).args
args

['self',
 'inputs',
 'preds',
 'return_vis',
 'show',
 'wait_time',
 'draw_pred',
 'pred_score_thr',
 'no_save_vis',
 'img_out_dir']

In [4]:
# inference
pcl = '../data/KITTI_lidar/testing/velodyne/'
# pcl = './data/falcon/falcon1.bin'

inputs = dict(points=pcl)
# inferencer(inputs, show=False,pred_score_thr=0.3, batch_size=4)

In [5]:
import time
start = time.time()
inferencer(inputs, show=False,pred_score_thr=0.3,batch_size=2)
end = time.time()
print("Time Taken {} ms".format((end-start)*1000))

Preprocessing took 97.84793853759766 ms
Voxel Encoder took 51.92255973815918 ms
Middle Encoder/Imagify took 15.018463134765625 ms
Backbone + FPN took 200.74939727783203 ms


/home/riccardo/venvs/pytorch_cuda123_env/lib/python3.10/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Bbox Head pred took 376.6789436340332 ms

Preprocessing took 56.319236755371094 ms
Voxel Encoder took 0.6341934204101562 ms
Middle Encoder/Imagify took 23.07915687561035 ms
Backbone + FPN took 1.0592937469482422 ms
Bbox Head pred took 65.92702865600586 ms

Preprocessing took 48.96140098571777 ms
Voxel Encoder took 1.6794204711914062 ms
Middle Encoder/Imagify took 31.2654972076416 ms
Backbone + FPN took 0.9922981262207031 ms
Bbox Head pred took 64.77046012878418 ms

Preprocessing took 57.01875686645508 ms
Voxel Encoder took 0.8955001831054688 ms
Middle Encoder/Imagify took 33.78033638000488 ms
Backbone + FPN took 1.1708736419677734 ms
Bbox Head pred took 65.14739990234375 ms

Preprocessing took 55.161476135253906 ms
Voxel Encoder took 1.45721435546875 ms
Middle Encoder/Imagify took 33.80250930786133 ms
Backbone + FPN took 1.1229515075683594 ms
Bbox Head pred took 65.35673141479492 ms

Preprocessing took 51.36585235595703 ms
Voxel Encoder took 0.9324550628662109 ms
Middle Encoder/Imagify

KeyboardInterrupt: 

In [6]:
from mmdet3d.datasets.transforms.loading import (LoadAnnotations3D,
                                                 LoadPointsFromFile)
from mmdet3d.models.data_preprocessors.data_preprocessor import \
    Det3DDataPreprocessor
from mmdet3d.datasets.transforms.formating import Pack3DDetInputs
import os 

model_inputs = {}
model_inputs["inputs"] = {}
model_inputs["inputs"]["points"] = []
max = 2

files = []
if not os.path.isdir(inputs["points"]):
    files.append(inputs["points"])
else:
    files = [inputs["points"] + s for s in os.listdir(inputs["points"])]
for i,sample_path in enumerate(files):
    inp = {}
    inp["lidar_points"] = {}
    inp["lidar_points"]["lidar_path"] = sample_path


    loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)
    packer = Pack3DDetInputs(keys=['points'])

    voxel_size = [0.16, 0.16, 4]
    preprocessor = Det3DDataPreprocessor(
        voxel=True,
        voxel_layer=dict(
            max_num_points=32,  # max_points_per_voxel
            point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
            voxel_size=voxel_size,
            max_voxels=(16000, 40000)))

    input = loader(inp)
    input = packer(input)
    # input["inputs"]["points"] = input["inputs"]["points"].cuda()
    model_inputs["inputs"]["points"].append(input["inputs"]["points"])
    if i == max-1:
        print("Max Size!")
        break
prep_input = preprocessor(model_inputs)



Max Size!
Preprocessing took 8.751153945922852 ms


In [7]:
prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()


In [3]:
model = inferencer.model

In [ ]:
prep_input["inputs"]["voxels"]["voxels"].shape

torch.Size([23358, 32, 4])

In [ ]:
import time 
import torch
s = time.time()
res = model(prep_input["inputs"],prep_input["data_samples"], mode="tensor")
print("Time: {} ms".format((time.time()-s)*1000))

Voxel Encoder took 7.731199264526367 ms
Middle Encoder/Imagify took 25.522470474243164 ms
Backbone + FPN took 16.983985900878906 ms
Bbox Head forward took 0.9174346923828125 ms

Time: 53.93028259277344 ms


## Model Analysis

In [4]:
model = inferencer.model

In [5]:
# Check the device of the model
device = next(inferencer.model.parameters()).device

if device.type == 'cuda':
    print("Model is on GPU")
else:
    print("Model is on CPU")

Model is on GPU


In [6]:
# Count the number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

# Access and print model parameters
for name, param in model.named_parameters():
    print(f"Parameter: {name}, Size: {param.size()}")

Total parameters: 4814868
Parameter: backbone.blocks.0.0.weight, Size: torch.Size([64, 64, 3, 3])
Parameter: backbone.blocks.0.1.weight, Size: torch.Size([64])
Parameter: backbone.blocks.0.1.bias, Size: torch.Size([64])
Parameter: backbone.blocks.0.3.weight, Size: torch.Size([64, 64, 3, 3])
Parameter: backbone.blocks.0.4.weight, Size: torch.Size([64])
Parameter: backbone.blocks.0.4.bias, Size: torch.Size([64])
Parameter: backbone.blocks.0.6.weight, Size: torch.Size([64, 64, 3, 3])
Parameter: backbone.blocks.0.7.weight, Size: torch.Size([64])
Parameter: backbone.blocks.0.7.bias, Size: torch.Size([64])
Parameter: backbone.blocks.0.9.weight, Size: torch.Size([64, 64, 3, 3])
Parameter: backbone.blocks.0.10.weight, Size: torch.Size([64])
Parameter: backbone.blocks.0.10.bias, Size: torch.Size([64])
Parameter: backbone.blocks.1.0.weight, Size: torch.Size([128, 64, 3, 3])
Parameter: backbone.blocks.1.1.weight, Size: torch.Size([128])
Parameter: backbone.blocks.1.1.bias, Size: torch.Size([128])

In [2]:
def prepare_input(inputs,batch_size=1):
    model_inputs = {}
    model_inputs["inputs"] = {}
    model_inputs["inputs"]["points"] = []
    max = batch_size

    files = []
    if not os.path.isdir(inputs["points"]):
        files.append(inputs["points"])
    else:
        files = [inputs["points"] + s for s in os.listdir(inputs["points"])]
    for i,sample_path in enumerate(files):
        inp = {}
        inp["lidar_points"] = {}
        inp["lidar_points"]["lidar_path"] = sample_path


        loader = LoadPointsFromFile(coord_type='LIDAR',load_dim=4,use_dim=4)
        packer = Pack3DDetInputs(keys=['points'])

        voxel_size = [0.16, 0.16, 4]
        preprocessor = Det3DDataPreprocessor(
            voxel=True,
            voxel_layer=dict(
                max_num_points=32,  # max_points_per_voxel
                point_cloud_range=[0, -39.68, -3, 69.12, 39.68, 1],
                voxel_size=voxel_size,
                max_voxels=(16000, 40000)))

        input = loader(inp)
        input = packer(input)
        # input["inputs"]["points"] = input["inputs"]["points"].cuda()
        model_inputs["inputs"]["points"].append(input["inputs"]["points"])
        if i == max-1:
            print("Batch Size!")
            break
    prep_input = preprocessor(model_inputs)
    prep_input["inputs"]["voxels"]["voxels"] = prep_input["inputs"]["voxels"]["voxels"].cuda()
    return prep_input

In [3]:
import torch
import os
from mmdet3d.datasets.transforms.loading import (LoadAnnotations3D,
                                                LoadPointsFromFile)
from mmdet3d.models.data_preprocessors.data_preprocessor import \
    Det3DDataPreprocessor
from mmdet3d.datasets.transforms.formating import Pack3DDetInputs

pcl_path = "../data/DonaSet/testing/velodyne/"
inputs = prepare_input({"points": pcl_path}, batch_size=3)["inputs"]
# Evaluate on validation data
model.eval()
with torch.no_grad():
    val_output = model(inputs,mode="tensor")


Batch Size!


NameError: name 'model' is not defined

In [4]:
import onnx

onnx_model_path = "end2end2.onnx"
m = onnx.load(onnx_model_path)
opset_version = m.opset_import[0].version if m.opset_import else None

print("ONNX Model Operator Set Version:", opset_version)

ONNX Model Operator Set Version: 11


In [5]:
import onnxruntime

ort_version = onnxruntime.__version__
print("ONNX Runtime Version:", ort_version)

print( onnxruntime.get_device()  )

ONNX Runtime Version: 1.15.1
GPU


In [6]:

import onnx
import onnxruntime
import onnxruntime as ort
import numpy as np

model_path = 'end2end2.onnx'
onnx_model = onnx.load(model_path)
onnx.checker.check_model(onnx_model)


# x, y = test_data[0][0], test_data[0][1]
ort_sess = ort.InferenceSession(model_path)
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}
outputs = ort_sess.run(None, onnx_inputs)

ValueError: This ORT build has ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'] enabled. Since ORT 1.9, you are required to explicitly set the providers parameter when instantiating InferenceSession. For example, onnxruntime.InferenceSession(..., providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'], ...)

In [61]:
val_output[0][0].shape,outputs[0].shape

(torch.Size([1, 2, 248, 216]), (1, 2, 248, 216))

In [15]:
# options = ort.SessionOptions()
# options.execution_mode = ort.ExecutionMode.ORT_SEQUENTIAL
# options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL

# # Specify the target device as CUDA if available
# # options.intra_op_num_threads = 1
# options.execution_mode = ort.ExecutionMode.ORT_SEQUENTIAL
# options.add_session_config_entry('session.device_id', '0')  # Set the GPU device ID

# Create the InferenceSession
ort_sess = ort.InferenceSession(model_path,providers=['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider'])

2024-02-20 21:08:29.326350419 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-20 21:08:29 WARNING] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
2024-02-20 21:08:29.326376271 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-20 21:08:29 WARNING] onnx2trt_utils.cpp:400: One or more weights outside the range of INT32 was clamped
2024-02-20 21:08:29.407009799 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-20 21:08:29 WARNING] onnx2trt_utils.cpp:374: Your ONNX model has been generated with INT64 weights, while TensorRT does not natively support INT64. Attempting to cast down to INT32.
2024-02-20 21:08:29.407033456 [W:onnxruntime:Default, tensorrt_execution_provider.h:75 log] [2024-02-20 21:08:29 WARNING] onnx2trt_utils.cpp:400: One or more weights outside the range of INT32 was clamped


In [16]:
onnx_inputs = {'voxels': inputs["voxels"]["voxels"].cpu().numpy(), 'coors':inputs["voxels"]["coors"].cpu().numpy(),'num_points':inputs["voxels"]["num_points"].cpu().numpy()}
outputs = ort_sess.run(None, onnx_inputs)

In [20]:
import time
s = time.time()
outputs = ort_sess.run(None, onnx_inputs)
print("Time: {} ms".format((time.time()-s)*1000))

Time: 26.1385440826416 ms


In [44]:
outputs[0].shape

(1, 2, 248, 216)

Test

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class MyModel(nn.Module):

    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
torch_model = MyModel()
torch_input = torch.randn(1, 1, 32, 32)
onnx_program = torch.onnx.dynamo_export(torch_model, torch_input)
onnx_program.save("my_image_classifier.onnx")

import onnx
onnx_model = onnx.load("my_image_classifier.onnx")
onnx.checker.check_model(onnx_model)

/home/riccardo/venvs/pytorch_cuda123_env/lib/python3.10/site-packages/torch/onnx/_internal/exporter.py:137: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


In [7]:
import onnxruntime

onnx_input = onnx_program.adapt_torch_inputs_to_onnx(torch_input)
print(f"Input length: {len(onnx_input)}")
print(f"Sample input: {onnx_input}")

ort_session = onnxruntime.InferenceSession("./my_image_classifier.onnx", providers=['CPUExecutionProvider'])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

onnxruntime_input = {k.name: to_numpy(v) for k, v in zip(ort_session.get_inputs(), onnx_input)}

onnxruntime_outputs = ort_session.run(None, onnxruntime_input)

Input length: 1
Sample input: (tensor([[[[ 0.4290,  0.0823,  0.3877,  ...,  0.2530, -1.5004,  0.2132],
          [ 0.0971, -1.2425,  1.5864,  ..., -0.5080,  0.2800,  0.0344],
          [-0.2636,  0.6221,  0.5165,  ...,  0.2748,  0.5340, -0.6011],
          ...,
          [ 0.3662, -1.2284,  2.0043,  ...,  0.9872,  0.5133,  0.5443],
          [-1.1936,  1.5664,  0.0353,  ...,  0.4518,  0.2531,  0.2016],
          [ 0.2531,  2.8630, -0.9375,  ..., -0.1628, -1.0180, -0.6954]]]]),)


Fail: [ONNXRuntimeError] : 1 : FAIL : Load model from ./my_image_classifier.onnx failed:/onnxruntime_src/onnxruntime/core/graph/model_load_utils.h:57 void onnxruntime::model_load_utils::ValidateOpsetForDomain(const std::unordered_map<std::__cxx11::basic_string<char>, int>&, const onnxruntime::logging::Logger&, bool, const string&, int) ONNX Runtime only *guarantees* support for models stamped with official released onnx opset versions. Opset 18 is under development and support for this is limited. The operator schemas and or other functionality may change before next ONNX release and in this case ONNX Runtime will not guarantee backward compatibility. Current official support for domain ai.onnx is till opset 17.


In [8]:
import numpy as np
path = '../data/DonaSet/testing/velodyne/000001.bin'

fields = 4
# Read the binary file into a NumPy array
data = np.fromfile(path, dtype=np.float32)

# Reshape the 1D array to a 2D array with 4 columns
data = data.reshape(-1, 4)
data[0]

array([ 17.639317  , -13.64231   ,   0.98981375,   0.        ],
      dtype=float32)

In [47]:
import torch 
from mmdet3d.structures import  Det3DDataSample
# Assuming you have a single sample as a NumPy array or list

# Convert the sample to a PyTorch tensor
input_tensor = torch.tensor(data, dtype=torch.float32)

# Add batch dimension (if needed)
input_tensor = input_tensor  # For models expecting a batch dimension

# Set the model to evaluation mode
model.eval()

model_input = {}
model_input["inputs"] = {}
model_input["inputs"]["points"] = [input_tensor]
data_sample = Det3DDataSample()
model_input["data_samples"] = [data_sample]
# Perform inference
with torch.no_grad():
    output = model.test_step(model_input)

NameError: name 'data' is not defined

In [53]:
# Print the input size of the first layer
input_size = next(model.parameters()).size()[1:]
print("Input size:", input_size)

Input size: torch.Size([64, 3, 3])


In [24]:
from mmdet3d.registry import DATASETS, MODELS
MODELS.build(mod)

/home/riccardo/LidarObjDetection/mmdetection3d_dona/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:94: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


AttributeError: 'dict' object has no attribute 'assigner'